In [120]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import implicit
import numpy as np
import scipy

df = pd.read_csv('lastfm_9000_users.csv', na_filter=False)

data = df.drop(['Unnamed: 0','artist_mbid'], axis=1)

users = list(np.sort(data.user_id.unique()))
artists = list(data.artist_name.unique())
plays = list(data.plays)

#user-user
rows = data.user_id.astype('category', categories=users).cat.codes
cols = data.artist_name.astype('category', categories=artists).cat.codes
plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(users),len(artists)))

#item-item
rows = data.artist_name.astype('category', categories=artists).cat.codes
cols = data.user_id.astype('category', categories=users).cat.codes
plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(artists),len(users)))

plays_sparse

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  app.launch_new_instance()
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:22: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


<53577x9000 sparse matrix of type '<class 'numpy.int64'>'
	with 438344 stored elements in Compressed Sparse Row format>

In [122]:
# initialize a model
model = implicit.als.AlternatingLeastSquares(factors=50)

In [123]:
# train the model on a sparse matrix of item/user/confidence weights
model.fit(plays_sparse)

100%|██████████| 15.0/15 [00:05<00:00,  2.66it/s]


In [140]:
#ignore this, don't run this for now
# recommend items for a user
userid = 1
user_items = plays_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)
recommendations
print("These are the artists are recommended to user ",users[userid])
for (x,y) in recommendations:
    print(artists[x])

These are the artists are recommended to user  000a1585c5f65532a9c9187a882892982d345a5c
kaiser chiefs
amy macdonald
beatsteaks
the killers
jason mraz
panic at the disco
snow patrol
franz ferdinand
james blunt
fall out boy


In [134]:
# find related items
dummy=34
related = model.similar_items(dummy)

In [135]:
print("These are the artists similar to ",artists[dummy])
for (x,y) in related:
    print(artists[x])

These are the artists similar to  cat power
cat power
beirut
devendra banhart
pj harvey
pixies
of montreal
feist
sufjan stevens
interpol
elliott smith
